# Ustawianie środowiska pod syllable-RNN

## Instalacja PyTorch

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

# !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision

import torch

## Instalacja bibliotek

In [0]:
!pip install -q scipy --upgrade

In [0]:
!pip install -q unidecode tqdm dill pysmi visdom

## Instalacja biblioteki PyDrive i wrappera

In [0]:
# This only needs to be done once per notebook.
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

from pathlib import Path
import os

def download_and_save(file_name, file_id):
  downloaded = drive.CreateFile({'id': file_id})
  # fetch file
  downloaded.FetchContent()
  # write file
  with open(file_name,'wb') as f:
       f.write(downloaded.content.read())
      
  print(f'Saved {file_name}')
  
FILE_NAME, FILE_ID = ['file_name', 'file_id']

[link text](https://)## Pobranie i instalacja stemmera

In [0]:
!dpkg --add-architecture i386
!apt-get -qq update
!apt-get -qq install libc6:i386 libncurses5:i386 libstdc++6:i386

In [0]:
# stemmer-2.0.3.tgz, https://drive.google.com/file/d/1PxS-e6tzXa7u4N9A2r9XWLUpvSKqJIrZ
stemmer = {FILE_NAME: 'stemmer-2.0.3.tgz', FILE_ID: '1PxS-e6tzXa7u4N9A2r9XWLUpvSKqJIrZ'}
stemmer_path = Path('./') / stemmer[FILE_NAME]
download_and_save(stemmer_path, stemmer[FILE_ID])

In [0]:
!tar xzf $stemmer_path

In [0]:
ls -lah bin

# Pobranie danych

In [0]:
from pathlib import Path
dataset_path = Path('/content/data/rnn_generator'); dataset_path
!mkdir -p $dataset_path
!ls -la $dataset_path/
tmp_path = dataset_path / 'tmp/'
!mkdir -p $tmp_path

## Pobranie datasetu Pan Tadeusz

In [0]:
# pan_tadeusz, https://drive.google.com/open?id=1l8KGrqUsrGOlXVp7Frgyku8w75X_WA8h
pan_tadeusz = {FILE_NAME: 'pan_tadeusz.txt', FILE_ID: '1l8KGrqUsrGOlXVp7Frgyku8w75X_WA8h'}

download_and_save(dataset_path / pan_tadeusz[FILE_NAME], pan_tadeusz[FILE_ID])

In [0]:
ls -la $dataset_path/

# Opcjonalne

## Opcjonalne pobranie innych datasetów

In [0]:
# witkacy_szewcy, https://drive.google.com/open?id=1NOWTocYwv93GibItaNBxTJobjTukQmRP
witkacy = {FILE_NAME: 'witkacy_szewcy.txt', FILE_ID: '1NOWTocYwv93GibItaNBxTJobjTukQmRP'}

download_and_save(dataset_path / witkacy[FILE_NAME], witkacy[FILE_ID])

In [0]:
# mickiewicz, https://drive.google.com/open?id=1otbOB7GjKEVPjaPShaaTnYQnv-whwNWP
mickiewicz = {FILE_NAME: 'mickiewicz.txt', FILE_ID: '1otbOB7GjKEVPjaPShaaTnYQnv-whwNWP'}

download_and_save(dataset_path / mickiewicz[FILE_NAME], mickiewicz[FILE_ID])

In [0]:
!ls -la $dataset_path/

## Opcjonalne pobranie przetrenowanego modelu

In [0]:
# all_tokens.n5059.pan_tadeusz.p, https://drive.google.com/open?id=14AxyK--ifqyzwThw712jHid7NVaLqAC3
# pan_tadeusz.h500.l3.20.gpu.torch, https://drive.google.com/open?id=1NF-bmLgsAC6l_Fdlr7xZ1_i3e_5EgIYG
ALLTOKS, MODEL = ['all_tokens', 'model']
fn_pan_tadeusz = {ALLTOKS: 'all_tokens.n5059.pan_tadeusz.p', MODEL: 'pan_tadeusz.h500.l3.e20.gpu.torch'}
ids_pan_tadeusz = {ALLTOKS: '14AxyK--ifqyzwThw712jHid7NVaLqAC3', MODEL: '1NF-bmLgsAC6l_Fdlr7xZ1_i3e_5EgIYG'}
download_and_save(tmp_path / fn_pan_tadeusz[ALLTOKS], ids_pan_tadeusz[ALLTOKS])
download_and_save(tmp_path / fn_pan_tadeusz[MODEL], ids_pan_tadeusz[MODEL])

In [0]:
# all_tokens.n4064.witkacy_szewcy.p, https://drive.google.com/open?id=1hXWaH1vwciJqDh4mRA9R1B6ePapORCTc
# witkacy_szewcy.h500.l3.e11.gpu.torch, https://drive.google.com/open?id=1DegovTSmaNEyFM3AS8JZZnp7STO49IRp
ALLTOKS, MODEL = ['all_tokens', 'model']
fn_pan_tadeusz = {ALLTOKS: 'all_tokens.n4064.witkacy_szewcy.p', MODEL: 'witkacy_szewcy.h500.l3.e11.gpu.torch'}
ids_pan_tadeusz = {ALLTOKS: '1hXWaH1vwciJqDh4mRA9R1B6ePapORCTc', MODEL: '1DegovTSmaNEyFM3AS8JZZnp7STO49IRp'}
download_and_save(tmp_path / fn_pan_tadeusz[ALLTOKS], ids_pan_tadeusz[ALLTOKS])
download_and_save(tmp_path / fn_pan_tadeusz[MODEL], ids_pan_tadeusz[MODEL])

In [0]:
ls -lah $tmp_path

## Opcjonalny monitoring treningu

In [0]:
!python -m visdom.server -p 8890 -logging_level ERROR

## Opcjonalna konfiguracja integracji z Dyskiem Google



In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive -o nonempty

# Monitorowanie maszyny wirtualnej

In [0]:
!uptime